# Install mysql connector

In [1]:
%pip install mysql-connector-python

   ---------------------------------------- 0.0/14.3 MB ? eta -:--:--
   ----- ---------------------------------- 2.1/14.3 MB 16.8 MB/s eta 0:00:01
   --------------------- ------------------ 7.9/14.3 MB 23.2 MB/s eta 0:00:01
   ---------------------------------------  14.2/14.3 MB 26.2 MB/s eta 0:00:01
   ---------------------------------------- 14.3/14.3 MB 24.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


# Test MySQL Templates

In [1]:
from mysql_template import MySQLTemplate

template = MySQLTemplate()
template.template_select()

{'query': 'SELECT * from current_dept_emp limit 5',
 'result': [(10001,
   'd005',
   datetime.date(1986, 6, 26),
   datetime.date(9999, 1, 1)),
  (10002, 'd007', datetime.date(1996, 8, 3), datetime.date(9999, 1, 1)),
  (10003, 'd004', datetime.date(1995, 12, 3), datetime.date(9999, 1, 1)),
  (10004, 'd004', datetime.date(1986, 12, 1), datetime.date(9999, 1, 1)),
  (10005, 'd003', datetime.date(1989, 9, 12), datetime.date(9999, 1, 1))],
 'message': 'success'}

In [2]:
template.template_limit()

{'query': 'SELECT * from employees limit 5',
 'result': [(10001,
   datetime.date(1953, 9, 2),
   'Georgi',
   'Facello',
   'M',
   datetime.date(1986, 6, 26)),
  (10002,
   datetime.date(1964, 6, 2),
   'Bezalel',
   'Simmel',
   'F',
   datetime.date(1985, 11, 21)),
  (10003,
   datetime.date(1959, 12, 3),
   'Parto',
   'Bamford',
   'M',
   datetime.date(1986, 8, 28)),
  (10004,
   datetime.date(1954, 5, 1),
   'Chirstian',
   'Koblick',
   'M',
   datetime.date(1986, 12, 1)),
  (10005,
   datetime.date(1955, 1, 21),
   'Kyoichi',
   'Maliniak',
   'M',
   datetime.date(1989, 9, 12))],
 'message': 'success'}